# Bowtie2 Optimization
```
Lead     : Charles
Issue    : #35 - Hard Optimize `bowtie2` alignment parameters
Start    : 2020 04 19
Complete : YYYY MM DD
Files    : ~/serratus/notebook/200419
```

## Introduction
We are currently running `bowtie2 --very-sensitive-local ...` for detecting homologous CoV sequences. We need a method (script) to test an array of `bowtie2` parameters for time, sensitivity and specificity of alignment.

Current settings are `-D 20 -R 3 -N 0 -L 20 -i S,1,0.50`, say we'd like to test the space of `-D 5-25 -R 1-4 -N 0-1 -L 30-15`.
### Objectives
Determine optimal `bowtie2` alignment parameters by outputting the following:

1) Wall-clock / CPU time / User time for each setting.
2) TP / FP / TN / FN



## Materials and Methods

- [bowtie2 manual](http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml)

Reads from simulated divergent genomes of SARS-CoV-2. Created by mutating the SARS-CoV-2 reference sequence with random subsitutions at different rates (0.1% - 40%).
Reads can be accessed via [Amazon S3](s3://serratus-public/notebook/200411/)



## Results & Discussion
